In [24]:
!pip install ucimlrepo

from ucimlrepo import fetch_ucirepo

# fetch dataset
breast_cancer_wisconsin_diagnostic = fetch_ucirepo(id=17)

# data (as pandas dataframes)
X = breast_cancer_wisconsin_diagnostic.data.features
y = breast_cancer_wisconsin_diagnostic.data.targets

# metadata
print(breast_cancer_wisconsin_diagnostic.metadata)

# variable information
print(breast_cancer_wisconsin_diagnostic.variables)

{'uci_id': 17, 'name': 'Breast Cancer Wisconsin (Diagnostic)', 'repository_url': 'https://archive.ics.uci.edu/dataset/17/breast+cancer+wisconsin+diagnostic', 'data_url': 'https://archive.ics.uci.edu/static/public/17/data.csv', 'abstract': 'Diagnostic Wisconsin Breast Cancer Database.', 'area': 'Health and Medicine', 'tasks': ['Classification'], 'characteristics': ['Multivariate'], 'num_instances': 569, 'num_features': 30, 'feature_types': ['Real'], 'demographics': [], 'target_col': ['Diagnosis'], 'index_col': ['ID'], 'has_missing_values': 'no', 'missing_values_symbol': None, 'year_of_dataset_creation': 1993, 'last_updated': 'Fri Nov 03 2023', 'dataset_doi': '10.24432/C5DW2B', 'creators': ['William Wolberg', 'Olvi Mangasarian', 'Nick Street', 'W. Street'], 'intro_paper': {'title': 'Nuclear feature extraction for breast tumor diagnosis', 'authors': 'W. Street, W. Wolberg, O. Mangasarian', 'published_in': 'Electronic imaging', 'year': 1993, 'url': 'https://www.semanticscholar.org/paper/53

In [26]:
import pandas as pd
import numpy as np

In [29]:
def train_test_split(X, y, test_size=0.2, random_state=None):
    if random_state is not None:
        np.random.seed(random_state)

    # Determine the type of the dataset
    is_pandas = isinstance(X, pd.DataFrame)

    # Shuffle the indices
    indices = np.arange(X.shape[0])
    np.random.shuffle(indices)

    # Split indices
    split_idx = int(len(indices) * (1 - test_size))
    train_idx, test_idx = indices[:split_idx], indices[split_idx:]

    # Split the data
    if is_pandas:
        X_train, X_test = X.iloc[train_idx], X.iloc[test_idx]
        y_train, y_test = y.iloc[train_idx], y.iloc[test_idx]
    else:
        X_train, X_test = X[train_idx], X[test_idx]
        y_train, y_test = y[train_idx], y[test_idx]

    return X_train, X_test, y_train, y_test


X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
print("X_train shape:", X_train.shape)
print("X_test shape:", X_test.shape)
print("y_train shape:", y_train.shape)
print("y_test shape:", y_test.shape)
print(y)




X_train shape: (455, 30)
X_test shape: (114, 30)
y_train shape: (455, 1)
y_test shape: (114, 1)
     Diagnosis
0            1
1            1
2            1
3            1
4            1
..         ...
564          1
565          1
566          1
567          1
568          0

[569 rows x 1 columns]


In [31]:
def standard_scaler(X_train, X_test):
  mean = np.mean(X_train, axis=0)
  std = np.std(X_train, axis=0)

  # Avoid division by zero in case of zero variance
  std = np.where(std == 0, 1, std)

  # Perform scaling
  X_train_scaled = (X_train - mean) / std
  X_test_scaled = (X_test - mean) / std

  return X_train_scaled, X_test_scaled



X_train_scaled, X_test_scaled = standard_scaler(X_train, X_test)
print("Original X_train mean:", np.mean(X_train, axis=0))
print("Scaled X_train mean:", np.mean(X_train_scaled, axis=0))



Original X_train mean: radius1                14.053226
texture1               19.311341
perimeter1             91.496571
area1                 647.105055
smoothness1             0.096316
compactness1            0.104691
concavity1              0.088904
concave_points1         0.048606
symmetry1               0.181284
fractal_dimension1      0.062977
radius2                 0.398979
texture2                1.232834
perimeter2              2.828935
area2                  38.947615
smoothness2             0.007021
compactness2            0.025552
concavity2              0.032247
concave_points2         0.011661
symmetry2               0.020617
fractal_dimension2      0.003799
radius3                16.203837
texture3               25.737363
perimeter3            106.930593
area3                 872.951429
smoothness3             0.132254
compactness3            0.254927
concavity3              0.272903
concave_points3         0.113689
symmetry3               0.291066
fractal_dimension3  

In [33]:
class SVM:
    def __init__(self, learning_rate=0.1, lambda_param=0.1, n_iters=1000):
        self.lr = learning_rate
        self.lambda_param = lambda_param
        self.n_iters = n_iters
        self.w = None
        self.b = 0

    def fit(self, X, y):
        n_samples, n_features = X.shape
        y_ = np.where(y <= 0, -1, 1)

        # Initialize weights
        self.w = np.zeros(n_features)

        # Gradient descent
        for _ in range(self.n_iters):
            for idx in range(n_samples):
                x_i = X[idx] if isinstance(X, np.ndarray) else X.values[idx]
                condition = y_[idx] * (np.dot(x_i, self.w) + self.b) >= 1
                # print(condition)
                if condition:
                    self.w -= self.lr * (2 * self.lambda_param * self.w)
                else:
                    self.w -= self.lr * (2 * self.lambda_param * self.w - x_i * y_[idx])
                    self.b -= self.lr * y_[idx]

    def predict(self, X):
        X = X if isinstance(X, np.ndarray) else X.values
        linear_output = np.dot(X, self.w) + self.b
        return np.sign(linear_output)

    def score(self, X, y):
        predictions = self.predict(X)
        y = y.values if isinstance(y, pd.Series) else np.array(y)
        return np.mean(predictions == y)


# Initialize and train the SVM
svm_model = SVM()
svm_model.fit(X_train_scaled, y_train)

# Evaluate the model
y_pred = svm_model.predict(X_test_scaled)
accuracy = svm_model.score(X_test_scaled, y_test)
print("Accuracy:", accuracy)


Accuracy: 0.41228070175438597


In [34]:
import numpy as np
from cvxopt import matrix, solvers

class SVMClassifier:
    def __init__(self, C=1.0, sigma=0.5):
        self.C = C
        self.sigma = sigma
        self.alpha_sv = None
        self.sv_x = None
        self.sv_y = None
        self.b = 0

    def gaussian_kernel(self, x, y):
        return np.exp(-np.linalg.norm(x-y)**2 / (2 * (self.sigma ** 2)))

    def compute_kernel_matrix(self, X1, X2=None):
        if X2 is None:
            X2 = X1
        K = np.zeros((X1.shape[0], X2.shape[0]))
        # print("X1 type", type(X1))
        # print("X2 type", type(X2))
        if (type(X1)!=np.ndarray):
          # print(type(X1))
          # print(X1)
          X1=X1.to_numpy()

        for i in range(X1.shape[0]):
            for j in range(X2.shape[0]):
                K[i, j] = self.gaussian_kernel(X1[i], X2[j])
        return K

    def solve_svm_problem(self, X, y):
        n_samples = X.shape[0]
        K = self.compute_kernel_matrix(X)

        P = matrix(np.outer(y, y) * K)
        q = matrix(-np.ones((n_samples, 1)))
        G = matrix(np.vstack((np.eye(n_samples)*-1, np.eye(n_samples))))
        h = matrix(np.hstack((np.zeros(n_samples), np.ones(n_samples) * self.C)))
        A = matrix(y.reshape(1, -1), (1, n_samples), 'd')
        b = matrix(0.0)

        solvers.options['show_progress'] = False
        solution = solvers.qp(P, q, G, h, A, b)

        alphas = np.array(solution['x']).flatten()
        return alphas

    def fit(self, X, y):
        X = X.to_numpy()
        y = y['Diagnosis'].to_numpy()
        y[y==0] = -1
        alphas = self.solve_svm_problem(X, y)
        sv = (alphas > 1e-4)
        indices = np.arange(len(alphas))[sv]
        self.alpha_sv = alphas[sv]
        self.sv_x = X[sv]
        self.sv_y = y[sv]
        print(type(self.sv_x))
        self.b = np.mean([y_k - np.sum(self.alpha_sv * self.sv_y * self.compute_kernel_matrix(self.sv_x, np.array([self.sv_x[i]])))
                          for i, y_k in enumerate(self.sv_y)])

    def predict(self, X):
        if self.sv_x is None:
            raise ValueError("The model has not been fitted yet!")

        K = self.compute_kernel_matrix(X, self.sv_x)
        y_predict = np.sum(self.alpha_sv * self.sv_y * K, axis=1) + self.b
        return np.sign(y_predict)




# Initialize and train the SVM
svm_model = SVMClassifier()
svm_model.fit(X_train_scaled, y_train)

# Evaluate the model
y_pred = svm_model.predict(X_test_scaled)
print(y_pred)
print(type(y_pred))
y_test = y_test['Diagnosis'].to_numpy()
print(y_test)
y_test[y_test==0] = -1
accuracy = np.mean(y_pred == y_test)
print("Accuracy:", accuracy)


<class 'numpy.ndarray'>
[-1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1.
 -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1.
 -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1.
 -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1.
 -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1.
 -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1.
 -1. -1. -1. -1. -1. -1.]
<class 'numpy.ndarray'>
[1 1 0 1 1 0 0 0 1 0 1 1 1 1 0 0 1 0 0 1 1 1 1 1 1 0 0 0 0 0 0 0 1 1 1 1 0
 0 0 0 1 0 1 0 0 0 0 0 1 1 1 0 0 1 0 0 1 1 1 1 0 0 1 1 0 0 0 1 1 1 0 0 1 0
 0 0 0 1 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 1 0 0 1 0 0 1 1 1 0 1 1 0 1 0 0 0
 0 1 0]
Accuracy: 0.5877192982456141


In [36]:
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score
from sklearn.preprocessing import StandardScaler
# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# It's good practice to scale features before feeding them to SVM
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Initialize and train the classifier with default parameters
svm_model = SVC(random_state=42)
svm_model.fit(X_train_scaled, y_train)
# Make predictions
y_pred = svm_model.predict(X_test_scaled)

# Evaluate the model
print("Accuracy:", accuracy_score(y_test, y_pred))
print(classification_report(y_test, y_pred))


Accuracy: 0.9824561403508771
              precision    recall  f1-score   support

           0       0.97      1.00      0.99        71
           1       1.00      0.95      0.98        43

    accuracy                           0.98       114
   macro avg       0.99      0.98      0.98       114
weighted avg       0.98      0.98      0.98       114



/usr/local/lib/python3.10/dist-packages/sklearn/utils/validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
